<a href="https://colab.research.google.com/github/Rockk21/Code/blob/main/Copy_of_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df= pd.read_csv(r'/content/contr&res.csv')
df.head()

,Unnamed: 0,document,label
0,0,exhibit operations maintenance agreement piñon...,contract
1,1,confidential treatment requested schedule prod...,contract
2,2,exhibit shares bluerock residential growth rei...,contract
3,3,finance manager summary detailed oriented fina...,resume
4,4,engineering project manager summary seeking lo...,resume


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42,stratify=df['label'])

In [ ]:
from transformers import BertTokenizer
import torch

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
train_encodings = tokenizer(train_data['document'].tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data['document'].tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')

# Define a label mapping
label_mapping = {'resume': 0, 'contract': 1}

# Apply label mapping to the 'label' column
train_data['label'] = train_data['label'].map(label_mapping)
test_data['label'] = test_data['label'].map(label_mapping)

# Convert labels to tensor (make sure they are int64)
train_labels = torch.tensor(train_data['label'].values, dtype=torch.long)
test_labels = torch.tensor(test_data['label'].values, dtype=torch.long)



In [ ]:
# Check if any labels weren't mapped
print(train_data['label'].isnull().sum())  # Should be 0
print(test_data['label'].isnull().sum())   # Should be 0

# Inspect unique values to confirm the label mapping worked
print(train_data['label'].unique())
print(test_data['label'].unique())


0
0
[0 1]
[1 0]


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Create Dataset objects
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

# Create DataLoader for batching
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
from transformers import BertForSequenceClassification, AdamW

# Load pre-trained BERT model with a classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Set up optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 20  # You can increase the number of epochs for better performance

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch in train_loader:
        # Unpack the batch and move to device
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()  # Clear gradients

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss  # Get the loss from outputs

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")


Epoch 1/20, Loss: 0.18025472602393064
Epoch 2/20, Loss: 0.01688095577345747
Epoch 3/20, Loss: 0.003009508333720819
Epoch 4/20, Loss: 0.0005178235566105555
Epoch 5/20, Loss: 0.0003426713778430389
Epoch 6/20, Loss: 0.0002551903741766216
Epoch 7/20, Loss: 0.0002015556884644967
Epoch 8/20, Loss: 0.0001664225990815019
Epoch 9/20, Loss: 0.00014161670818187608
Epoch 10/20, Loss: 0.0001224520081585204
Epoch 11/20, Loss: 0.00010582866471614343
Epoch 12/20, Loss: 9.350630635809567e-05
Epoch 13/20, Loss: 8.39302226301731e-05
Epoch 14/20, Loss: 7.428409406673852e-05
Epoch 15/20, Loss: 6.735932173022e-05
Epoch 16/20, Loss: 6.10520919845267e-05
Epoch 17/20, Loss: 5.654354552259772e-05
Epoch 18/20, Loss: 5.196318818855582e-05
Epoch 19/20, Loss: 4.7954433790032956e-05
Epoch 20/20, Loss: 4.529822985609321e-05


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluation function
def evaluate(model, test_loader):
    model.eval()  # Set model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():  # Disable gradient calculations
        for batch in test_loader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

# Evaluate the model on the test set
predictions, true_labels = evaluate(model, test_loader)

# Print classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=['resume', 'contract']))

print("Confusion Matrix:")
print(confusion_matrix(true_labels, predictions))


Classification Report:
              precision    recall  f1-score   support

      resume       1.00      1.00      1.00       113
    contract       1.00      1.00      1.00       102

    accuracy                           1.00       215
   macro avg       1.00      1.00      1.00       215
weighted avg       1.00      1.00      1.00       215

Confusion Matrix:
[[113   0]
 [  0 102]]


In [ ]:

output_dir=r'C://Users//Administrator//Documents//New folder'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


print(f"Model and tokenizer saved to {output_dir}")

Model and tokenizer saved to C://Users//Administrator//Documents//New folder


In [ ]:
# Load the fine-tuned model and tokenizer
model_path = r'C:\Users\Administrator\Documents\New folder'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

In [ ]:
from transformers import BertTokenizer
import torch
import torch.nn.functional as F

# Assuming `doc` contains your new document as a string
doc = """AGREEMENT ON THE NON-DISCLOSURE
OF CONFIDENTIAL INFORMATION (TWO-WAY)
This Agreement is made with an effective date of the day of 200_
between
The University of Bristol whose registered address is Senate House, Tyndall Ave, Bristol, BS8 1TH,
and ____________________________________________________________
 whose registered address is
 _________________________________ (the "Parties")
THE PURPOSE of this Agreement is to regulate the exchange and subsequent treatment of confidential
information to be received by or disclosed to the signatories to this Agreement, in the field of
 , so as to protect the proper interests of the disclosing
party whilst this confidential information is in the possession or control of the receiving party. For the purposes
of this Agreement the term confidential information includes proprietary materials and information relating
thereto including without limitation specifications, drawings, designs, computer software and knowhow.
In general the receiving party must afford disclosed confidential information the same degree of protection as it
would afford its own.
NOW IT IS HEREBY AGREED:-
1. The disclosure of confidential information is for the specific purpose of evaluating technology in the field
described above in the first instance, and will normally be between of
the University of Bristol and of . Any specific
documents or materials which are necessarily provided on loan for the above purposes will be specified
in a schedule to this Agreement and the receiving party will return these and any other documents or
materials subsequently provided to the disclosing party on request.
2. The parties will mark or otherwise designate confidential information to show expressly or by necessary
implication that it is imparted in confidence.
3. The receiving party will receive all confidential information (whether recorded in writing or by other
means or given orally without record) which is disclosed in connection with this Agreement subject to the
following conditions:
a) it will take all proper and reasonable measures to ensure that the confidentiality of such
information is maintained;
b) it will not use the information for any commercial purpose or manufacture without obtaining a
written licence or other agreement from the disclosing party;
c) it will not disclose the information to any third party without written permission;
d) it will not disclose the information to employees other than those above except to the extent
necessary to fulfil the purposes set out above and all such other employees to which it will
disclose it will be made aware of the confidential nature of the information, and the conditions of
disclosure herein;
e) it will not make any copy of or abstract of the information without specific written permission from
the disclosing party;
f) it will acknowledge the source (i.e. one of the organisations signatory to this Agreement) of, and
will mark "Confidential", any drawing, document or software incorporating the information.
4. Under the terms of this Agreement there is no explicit or implied transfer of ownership to the receiving
party of any drawings, documents or software, or the copyright subsisting in them.
PROVIDED that the obligations herein undertaken will not apply to:
a) information which at the time of disclosure is in the public domain or which after disclosure
becomes part of the public domain through no fault of the recipient, or
b) information which the recipient party can show was in its possession at the time of disclosure or
which is independently developed by the recipient and was not acquired directly or indirectly from
the disclosing party, or
c) information which is made public at any time by the disclosing party, or by others with the
permission of the disclosing party, or
d) information which is received by the receiving party from a third party without similar
restriction and without breach of this Agreement.
e) information which is required to be disclosed by legal process, law or regulatory authority.
5. Both parties agree that at all times, during and after the current discussions, and thereafter for a period
of ten (10) years, starting from the effective date of this Agreement, not to communicate or to divulge to
third parties confidential information received from the other party.
6. This Agreement is to be construed and enforced in accordance with English Law and is subject to the
exclusive jurisdiction of the English courts to which the parties hereto submit. This clause shall not
prevent a party from seeking interim relief in any court of competent jurisdiction.
Signed for and on behalf of the University of Bristol
______________________________________________Date___________________
Name in block letters ____________________________________________________
Signed for and on behalf of
______________________________________________Date___________________
Name in block letters ___________________________________________________"""

# Load the fine-tuned model and tokenizer
model_path = r'C:\Users\Administrator\Documents\New folder'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Step 1: Tokenize the new document
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoding = tokenizer(doc, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Step 2: Use the fine-tuned model to make a prediction
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(**encoding)

# Step 3: Get the logits and apply softmax to get probabilities
logits = outputs.logits
probs = F.softmax(logits, dim=1)  # Apply softmax to get probabilities

# Step 4: Find the predicted label
predicted_label = torch.argmax(probs, dim=1).item()  # Get the index of the highest probability

# Mapping back the label to the class
label_mapping = {0: 'resume', 1: 'contract'}
predicted_class = label_mapping[predicted_label]

# Output the result
print(f"The document is classified as: {predicted_class}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The document is classified as: contract


In [ ]:
doc="""CONSULTANT
Summary
Asa proud Microsoftemployee, I'mdriven byCustomer obsession, GrowthMindsetand One Microsoft principles. In every position I've had at
Microsoft, my guiding principle has been taking customer feedback and advocating forcross-organizationalinitiatives to unitetheecosystemto
providethe best possible product, servicesand supportexperiencefor ourcustomers. Combiningmy technical knowledgeand experience withmy
excellentcommunication, problemsolving and strategic planning skills, I have been on thefront lines providing dedicated engineering support;
delivered and created Services global portfolio offerings; architected and delivered cloud migration engagements; driven our critical watch-list
customers throughGet to Green programsand built proactive outreach scale models to empower theentire Microsoftecosystemfromaccount
teams, support, servicesand productengineering teams with visibility,awareness, dataand actionable guidanceto improvecustomers' Skypefor
businessand O365 experience.
Experience
02/2016 to Current
Consultant
Enterprise Communications GlobalPractice), Microsoft Accepted a position inECGP, specifically to help develop and expand the"Get
Healthy"programas wellasactasa Solutions Architect for pre-sales,artefactand delivery activities on theseengagements.
My key contribution to date has been taking the on premises framework and contentand developing the hybrid and Onlineframeworksand
content.
The programwill go liveat theend ofJuly at which point our focus will bearound training global delivery resources.
I have worked on PGescalations with largecustomers like 3Min their Online"Get Healthy"efforts,as wellasengagingwith customers like
Alcoa(SfBOnline), Aon (O365Dedicated), Lubrizol, AllState(on premises) to scope,customizeand deliver"Get Healthy"offerings
Assisted withAirLift deliveriesas wellasconducting technicalinterviews for Consultantand Architect levelcandidates.
06/2013 to 02/2016
Premier Field Engineer City , State
DSEworkload ofat least 1600 hoursas wellasassisting onmany transactionalengagements, specificallyRisk Assessmentasa Serviceand
Office 365 Network Performanceassessments forcustomers like Delta Airlines.
I worked with two dedicated customers,and many transactionalcustomers to analyze performanceissues,and assist with getting their
environments healthy usingRaaS, but primarily ongoingKHIand CQManalysisand SCOMalert tuning,and helping themmoveto more
real-time methods such as StatsMan and CallQualityDashboard As part ofthe AIGO365 teamwho recently achieved a 40 million dollar
O365 deal, I was the Lync DSE on the AIGaccount for two yearsand they havejust renewed their Lync DSE contract foranother two
years.
I focused on helping themstabilizetheirenvironmentand starting to position themfor successful hybrid environmentsas part oftheir O365
strategy.
I worked as the DSE for multiple othercustomers helping themtransition fromIM&P to replacingAT&T conferencing company-wide
(with enterprise voicecapabilities,analysisand correction ofall performanceand call quality issues,as wellasassisting themwith
merger/acquisition scenariosand successfulconsolidation/migration oftwo Lyncenvironments Assisted on transactionalengagements for
Citrix, CarnivalCruise Lines, Florida Hospital, Diebold, Kelloggs Company and provided back up for other PFEscoveringAmazon, Visa,
Mastercard, Humana, Deloitte, KingCounty and State ofWashington.
Volunteered for MySkillsForAfrika,and assisted the BotswanaInnovationHub with their Skypefor Business deployment.
01/2009 to 06/2013
Lead ApplicationEngineer CompanyName
Responsiblefor designing and engineering unified communication solutions.
Lead Lync ApplicationEngineer.
Completed migration fromOCS 2007 R2 300 user deployment to on-premise Lync 2010 deployment for 25K usercapacitywith IM,
presence, group chat, federation, PIC,audio-video collaboration and Cisco/Tandberg integration functionality.
Also completed acquisition and integration ofMorganKeegan's Lync 2010 environment.
Designed Lync 2013 architectureand completed PolycomintegrationwithRMXand DMAinfrastructure.
Additionalresponsibilities include mobile device managementsolutions deign and implementations, includingBlackberry, Good, ActiveSync
and MobileIron.
Education and Training
1998
Associate degree University ofStellenbosch and NelsonMandela MetropolitanUniversity
1996
MatriculationCertificate Collegiate Girls High School
07 / 2018
MCSE :CommunicationCommunication
Microsoft Â· License:F372-7058 MCSA:Office 365 - 07 / 2015 - 07/2018 Microsoft Â· License: 10013480
Skills
go live, Architect,audio, Cisco, Citrix, Consultant,contentand developing,content, delivery, designing, Engineer, focus, Hub, Innovation, Office,
migration, Enterprise, Network, Quality, real-time, Risk Assessment, sales, strategy, video
AdditionalInformation
MCITP:Lync Server Administrator 2010 - 05 / 2013 Microsoft Â· License: 1001348"""

# Step 1: Tokenize the new document
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoding = tokenizer(doc, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Step 2: Use the fine-tuned model to make a prediction
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(**encoding)

# Step 3: Get the logits and apply softmax to get probabilities
logits = outputs.logits
probs = F.softmax(logits, dim=1)  # Apply softmax to get probabilities

# Step 4: Find the predicted label
predicted_label = torch.argmax(probs, dim=1).item()  # Get the index of the highest probability

# Mapping back the label to the class
label_mapping = {0: 'resume', 1: 'contract'}
predicted_class = label_mapping[predicted_label]

# Output the result
print(f"The document is classified as: {predicted_class}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


The document is classified as: resume


In [ ]:
import streamlit as st
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the fine-tuned model and tokenizer
model_dir= r'C:\Users\Administrator\Documents\New folder'
model = BertForSequenceClassification.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Set model to evaluation mode
model.eval()

# Function to classify document
def classify_document(document):
    # Tokenize the input document
    encoding = tokenizer(document, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**encoding)
    # Get the logits and apply softmax
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_label = torch.argmax(probabilities, dim=1).item()

    # Label mapping
    label_mapping = {0: 'resume', 1: 'contract'}
    return label_mapping[predicted_label]

# Streamlit application layout
st.title("Document Classification App")
st.write("Upload a document to classify it as either a 'contract' or a 'resume'.")

# File uploader
uploaded_file = st.file_uploader("Choose a text file", type='txt')

if uploaded_file is not None:
    # Read the uploaded document
    document = uploaded_file.read().decode('utf-8')

    # Classify the document
    result = classify_document(document)

    # Display the result
    st.write(f"The uploaded document is classified as: **{result}**")
